# MAp: Mutation Accumulation (in plasmids)

This code simulates a mutation accumulation experiment in bacterial cells that carry multiple copies of a plasmid. The simulation models the growth of a bacterial population over multiple generations, with each generation consisting of discrete time steps. Plasmid replication is modeled by replicating plasmids randomly until reaching the maximum plasmid copy number (PCN), and plasmid segregation is modeled by randomly partitioning plasmids to each daughter cell during cell division, with a probability of 0.5.

The simulation tracks the number and identity of plasmids carried by each cell in the population over multiple generations. Mutations can occur during plasmid replication, leading to new plasmid variants with different properties. The simulation allows for different mutation rates and maximum numbers of plasmids per cell and can be run for multiple replicates to assess the stochasticity of the evolutionary process. The code generates output files containing the plasmid composition of each cell in the population over time, which can be analyzed to study the dynamics of plasmid evolution in bacterial populations. The code can be run in parallel to speed up simulations for different parameter values.

The goal of this script is to simulate the dynamics of plasmid evolution in bacterial populations by comparing the opposing forces: an increase in the probability of mutation with an increase in PCN, and the enhanced probability of losing plasmids through segregational drift at higher PCN.

In [1]:
import copy
from datetime import datetime
import random
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
import os
from matplotlib.ticker import MaxNLocator
from collections import Counter
import time
import re
import sys
import pickle

## User-defined parameters

In [2]:

# Define the base path to the shared project directory
base_path = '/content/drive/MyDrive/SYNC_Projects/MAp/'
expe='test/' #'sim_dev_18e-7/'

# num_generations: number of generations in the simulation
num_generations = 24

# num_reps: number of replicate simulations to run for each parameter value
num_reps = 10

# mut_rate: mutation rate per plasmid per generation
mut_rate = 18e-7
#mut_rate = 1e-3

# num_days: number of days of simulated bacterial growth
num_days = 60

# maxexpe: maximum number of simulations before aborting
maxexpe = 1e6

# max_plasmids: a list of maximum plasmid copy numbers to simulate
#max_plasmids = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]
max_plasmids = [1, 5, 10, 50, 100]


### Connect with Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define relative paths from the base path
pathDATA = os.path.join(base_path, 'data/')
pathSIM = os.path.join(base_path, 'code/py-MAp/runs/')
pathCODE = os.path.join(base_path, 'code/py-MAp/')
sys.path.append(os.path.join(pathCODE))

# Verify paths and imported data
print("pathDATA:", pathDATA)
print("pathSIM:", pathSIM)
print("pathCODE:", pathCODE)

# Import functions
import MAp

Mounted at /content/drive
pathDATA: /content/drive/MyDrive/SYNC_Projects/MAp/data/
pathSIM: /content/drive/MyDrive/SYNC_Projects/MAp/code/py-MAp/runs/
pathCODE: /content/drive/MyDrive/SYNC_Projects/MAp/code/py-MAp/
['Bacteria', 'Counter', 'Plasmid', 'Population', 'Rectangle', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'column', 'datetime', 'getCountFixedMuts', 'getCountLostMuts', 'getCountMuts', 'getCountNewMuts', 'getMutGenerations', 'importData', 'np', 'plotCountMutGenerations', 'plotMutDays', 'plotMutGenerations', 'plt', 'random', 'time']


# Simulation functions

The simulateTransfer() function is designed to model the dynamics of plasmids within bacterial populations over a set number of days and generations. This function simulates a serial dilution experiment, typically involving the transfer of a single bacterial cell after a specified number of generations. The parameters include the number of days (num_days), the number of generations per day (num_generations), the maximum number of plasmids a bacterium can carry (max_plasmids), the mutation rate (mut_rate), and the output directory (dirSIM). An optional parameter (irep) is used for specifying the iteration number.

The function initiates by creating a single bacterium (B0) with given parameters like mutation rate and maximum plasmid count. A list of plasmids is generated and associated with this bacterium. This initial bacterium is then used to instantiate a population object (pop).

As the simulation progresses, it iterates through the designated number of days and generations per day. Within each cycle, the grow() method of the Population class is invoked, performing tasks like plasmid replication and segregation for each bacterium in the population. Segregation events, if any, are recorded. After every generation, the relevant data is written to a text file.

At the end of each day, the bacterial population undergoes a transfer, retaining only one bacterium to initiate the population for the next day. If the final population consists entirely of wild-type bacteria (no mutations), the output file is deleted and the simulation is re-run.

The function ultimately returns the number of unsuccessful experiments, which are those that ended with a wild-type population (absence of mutations and segregation events).

In [4]:
def simulateTransfer(num_days, num_generations, max_plasmids, mut_rate, dirSIM='', irep=0):
    """
    Simulate the transfer of bacterial cells over several days.

    Parameters:
        num_days: Number of days to simulate.
        num_generations: Number of generations per day.
        max_plasmids: Maximum number of plasmids in each bacteria.
        mut_rate: Mutation rate.
        dirSIM: Directory for saving simulation data.
        irep: Iteration replicate number.
    """

    # Initialize parameters for Bacteria class
    this_id = 'B0'
    this_parameters = {
        "mut_rate": mut_rate,
        "max_plasmids": max_plasmids,
    }

    # Create a list of Plasmid objects
    plasmids = [MAp.Plasmid() for _ in range(max_plasmids)]

    # Initialize Bacteria object
    B0 = MAp.Bacteria(this_id, this_parameters)
    B0.plasmids = plasmids

    # Simulate plasmid behavior
    B0.simulate_plasmids()

    # Create Population object
    pop = MAp.Population([B0])

    # File path for saving simulation data
    fileSIM = f"{dirSIM}data/MAp_days{num_days}_gens{num_generations}_mut{mut_rate}_maxp{max_plasmids}_irep{irep}.txt"

    with open(fileSIM, 'w') as f:
        for day in range(num_days):
            for ti in range(num_generations):
                str_mut = pop.grow()

                if pop.is_segregant():  # No plasmids found
                    return 0

                # Write simulation data to file
                str_row = f"{day};{ti};{str_mut}"
                f.write(str_row)

            # Reset population for the next day
            Bi = pop.bacteria[0]
            pop = MAp.Population([Bi])

    # Check for mutations and remove file if none found
    wt = pop.isWT()
    if wt == 1:
        os.remove(fileSIM)
        return 0

    return 1


The **runExperiment()** function performs simulation experiments focused on bacterial and plasmid dynamics. It accepts two parameters: __max_plasmids__, which specifies the maximum number of plasmids that a single bacterium can have, and __verbose__, an optional Boolean flag that toggles progress messages during the simulation.

Inside the function, two counters (__irep__ for repetitions and __nexpe__ for the number of experiments) are initiated along with an empty list (__experiment_results__) to collect results.

In each loop iteration:

1.   It checks if a simulation file corresponding to the current set of parameters already exists. If it does, the function loads the file; otherwise, it triggers a new simulation via **simulateTransfer()**.
2.   If at least one mutation occurred during the simulation, the function processes the results to extract various statistics about mutations.
3.   These statistics are recorded into a dictionary object and appended to __experiment_results__.

At the end of all iterations, the variable __experiment_results__ list is saved to a .pkl file and returned for future analysis.

In [5]:

def runExperiments(this_max_plasmids, verbose=False):


    # Start timer
    start_time = time.time()

    # Initialize variables
    experiment_results=[]
    irep=0
    nexpe=0

    # Loop through repetitions and experiments
    while irep<num_reps and nexpe<maxexpe:
        # Create string pattern for simulation file name
        str_pattern='MAp_days'+str(num_days)+'_gens'+str(num_generations)+'_mut'+str(mut_rate)+'_maxp'+str(this_max_plasmids)+'_irep'+str(irep)+'_'

        # Check if simulation file exists
        this_nexpe=0
        dir = dirSIM+"/data/"
        for filepath in os.listdir(dir):
            if filepath.startswith(str_pattern):
                this_nexpe = int(re.findall('[0-9]+', filepath)[-1])

        if this_nexpe==0: #Simulation not found
            # Run simulation if file not found
            fileSIM=dirSIM+'data/MAp_days'+str(num_days)+'_gens'+str(num_generations)+'_mut'+str(mut_rate)+'_maxp'+str(this_max_plasmids)+'_irep'+str(irep)+'.txt'
            eureka_n=simulateTransfer(num_days, num_generations, this_max_plasmids, mut_rate, dirSIM, irep)
            if eureka_n>0:
                # Print message if verbose
                print('.', end = '')
                if verbose:
                    print("  %s: Simulation executed (nexpe=%s)"%((irep+1), (nexpe+1)))

        else: #Simulation found
            # Load simulation file if found
            fileSIM=dirSIM+'data/MAp_days'+str(num_days)+'_gens'+str(num_generations)+'_mut'+str(mut_rate)+'_maxp'+str(this_max_plasmids)+'_irep'+str(irep)+'_n'+str(this_nexpe)+'.txt'
            eureka_n=this_nexpe
            #irep=irep+1
            nexpe=this_nexpe
            if verbose:
                print('  %s: Simulation loaded (nexpe=%s)'%((irep+1), this_nexpe))

        if eureka_n>0: #Only if there was at least one mutation.
            # Import data from simulation file
            data_plasmids=MAp.importData(fileSIM)

            # Extract mutation data
            all_gens=range(0,num_generations*num_days)
            rep_muts, rep_fixed_muts=MAp.getMutGenerations(data_plasmids, all_gens)

            # Calculate mutation statistics
            rep_results={}
            rep_results['rep_count_muts']=MAp.getCountMuts(rep_muts)
            rep_results['rep_count_fixed_muts']=MAp.getCountFixedMuts(rep_fixed_muts)
            rep_results['rep_count_ht_muts']=rep_results['rep_count_muts']-rep_results['rep_count_fixed_muts']
            rep_results['rep_count_new_muts']=MAp.getCountNewMuts(rep_muts)
            rep_results['rep_cum_new_muts']=np.cumsum(rep_results['rep_count_new_muts'])
            rep_results['rep_count_lost_muts']=MAp.getCountLostMuts(rep_muts)
            rep_results['rep_cum_lost_muts']=np.cumsum(rep_results['rep_count_lost_muts'])

            # Update counters
            irep=irep+1
            #nexpe=nexpe+eureka_n

            nexpe=nexpe+1
            #print("    irep=%s    nexpe=%s "%(irep, nexpe))

            rep_results['nexpe']=nexpe

            experiment_results.append(rep_results)

            if this_nexpe==0:
              fileSIM_expe=(os.path.splitext(fileSIM)[0]+'_n'+str(nexpe)+'.txt')
              os.rename(fileSIM, fileSIM_expe) #Rename file to store number of experiments performed

              #this_nexpe = fileSIM_expe[fileSIM_expe.index('_n') + len('_n') : fileSIM_expe.index('.txt')] #Recover nexpe from file name
              #print(this_nexpe)
              nexpe=0
        else:
          nexpe=nexpe+1
          #if nexpe%1e3==0:
          #  print('.', end = '')
          #  if nexpe%1e5==0:
          #    print(' ',nexpe)

    filePKL_expe=dirSIM+'data/MAp_days'+str(num_days)+'_gens'+str(num_generations)+'_mut'+str(mut_rate)+'_maxp'+str(this_max_plasmids)+'.pkl'
    # Save the experiment_results to a .pkl file
    with open(filePKL_expe, 'wb') as f:
        pickle.dump(experiment_results, f)
        print('Saving %s'%filePKL_expe)

    return experiment_results


In [6]:
dirSIM=pathSIM+expe
#dirSIM='run' #uncomment to save data in new directory

if not os.path.exists(dirSIM):
  if not dirSIM:
    dt = datetime.now()
    now=dt.strftime("%H%M%S%f") #ID: Timestamp
    dirSIM=pathSIM+'_sim_%s/'%(now)

  os.mkdir(dirSIM)
  os.mkdir(dirSIM+'data/')
  os.mkdir(dirSIM+'figures/')
  print('mkdir '+dirSIM)

## Run Simulation

In [7]:


sim_results=[]
if True: #To run, or not to run
  for this_max_plasmids in max_plasmids:
    print("*** max_plasmids=",this_max_plasmids)
    experiment_results=runExperiments(this_max_plasmids, True)
    sim_results.append(experiment_results)

*** max_plasmids= 1
  1: Simulation loaded (nexpe=67)
.  2: Simulation executed (nexpe=525)
.  3: Simulation executed (nexpe=281)
.  4: Simulation executed (nexpe=87)
.  5: Simulation executed (nexpe=192)
.  6: Simulation executed (nexpe=809)
.  7: Simulation executed (nexpe=285)
.  8: Simulation executed (nexpe=730)
.  9: Simulation executed (nexpe=996)
.  10: Simulation executed (nexpe=360)
Saving /content/drive/MyDrive/SYNC_Projects/MAp/code/py-MAp/runs/test/data/MAp_days60_gens24_mut1.8e-06_maxp1.pkl
*** max_plasmids= 5
.  1: Simulation executed (nexpe=305)
.  2: Simulation executed (nexpe=941)
.  3: Simulation executed (nexpe=297)
.  4: Simulation executed (nexpe=51)
.  5: Simulation executed (nexpe=493)
.  6: Simulation executed (nexpe=72)
.  7: Simulation executed (nexpe=11)
.  8: Simulation executed (nexpe=105)
.  9: Simulation executed (nexpe=849)
.  10: Simulation executed (nexpe=119)
Saving /content/drive/MyDrive/SYNC_Projects/MAp/code/py-MAp/runs/test/data/MAp_days60_gens24